In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

データの読み込み

In [3]:
df_sub = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

データの中身を一部確認

In [4]:
df_train.head()

データの情報をdf.info()で確認できる

In [5]:
df_train.info()

isnull()：データが欠損値（空の場合）True判定<br>
sum()：Trueの数を数える

In [7]:
df_train.isnull().sum().head(100)

df.describe()：各列の特徴がわかる，count（データの数）, mean（平均）, std（標準偏差）など

In [8]:
df_train.describe()

In [9]:
missing = df_train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [10]:
df_train.isnull().sum()

GrLivAreaをx軸、SalePriceをy軸としてプロット<br>
正の相関が見える、外れ値も確認できる<br>
ちなみにGrLivAreaとはリビングの広さらしい、何故リビングの広さに着目したのか<br>
直観か？

In [11]:
fig, ax = plt.subplots()
ax.scatter(x = df_train["GrLivArea"], y = df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

seabornのdistplotメソッドでヒストグラムを作成<br>
引数はデータフレーム型

In [12]:
ax = sns.distplot(df_train["SalePrice"])

In [13]:
ax = sns.boxplot(x = df_train["SalePrice"])

外れ値の除去<br>
リビングが広すぎるおよび家賃が30万未満のデータは外れ値であると判定

In [14]:
df_train = df_train.drop(df_train[(df_train["GrLivArea"]>4000) & (df_train["SalePrice"]<300000)].index)

外れ値が消えたことを確認

In [15]:
fig, ax = plt.subplots()
ax.scatter(df_train["GrLivArea"], df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

In [16]:
ax = sns.boxplot(x=df_train["SalePrice"])

In [17]:
ax = sns.distplot(df_train["SalePrice"])

ntrain：訓練データの形状

In [18]:
ntrain = df_train.shape
display(ntrain)

各列において欠損値の割合を求める<br>
各列において欠損値の割合が0のカラムは削除

In [19]:
df_train_nu = (df_train.isnull().sum()/len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(100)

In [20]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation="90")
sns.barplot(x=df_train_nu.index, y=df_train_nu)
plt.xlabel("Features", fontsize=15)
plt.ylabel("Percent of missing values", fontsize=15)
plt.title("Percent missing data by feature", fontsize=15)

In [31]:
df_train.info()

In [32]:
df_train["PoolQC"].isnull().sum()

In [23]:
df_train["MiscFeature"].isnull().sum()

In [24]:
df_train["Fence"].isnull().sum()

In [30]:
df_train["FireplaceQu"].isnull().sum()

In [29]:
df_train.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1, inplace=True)

In [33]:
df_train.head()

In [34]:
df_train.isnull().sum()

2回目<br>
各列でデータの欠損率を算出

In [35]:
df_train_nu = (df_train.isnull().sum() / len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(30)

In [36]:
df_train.LotFrontage

LotFrontage：物件に面している通りの直線距離<br>
グラフの形がSalePriceと似ている

In [37]:
ax = sns.distplot(df_train["LotFrontage"])

In [38]:
ax = sns.boxplot(x=df_train["LotFrontage"])

LotFrontageの欠損値をデータの中央値で埋める

In [39]:
df_train["LotFrontage"].fillna(df_train.LotFrontage.median(), inplace=True)

In [40]:
ax = sns.distplot(df_train["LotFrontage"])

In [41]:
ax = sns.boxplot(x=df_train["LotFrontage"])

Garage：車庫もしくは作業場

In [42]:
df_train.GarageType

In [43]:
df_train["GarageType"].isnull().sum()

In [44]:
df_train["GarageType"].fillna(df_train["GarageType"].mode()[0], inplace=True)

In [45]:
df_train["GarageType"].isnull().sum()

GarageYrBlt：GarageYearBuilt、ガレージの建築年

In [46]:
df_train.GarageYrBlt

In [47]:
ax = sns.distplot(df_train["GarageYrBlt"])

In [48]:
df_train["GarageYrBlt"].isnull().sum()

In [49]:
df_train.GarageYrBlt.median()

In [50]:
df_train["GarageYrBlt"].fillna(df_train.GarageYrBlt.median(), inplace=True)

In [51]:
df_train["GarageYrBlt"].isnull().sum()

In [52]:
df_train.GarageFinish

In [53]:
df_train["GarageFinish"].fillna(df_train.GarageFinish.mode()[0], inplace=True)

In [54]:
df_train["GarageFinish"].isnull().sum()

In [55]:
df_train["GarageQual"]

In [56]:
df_train["GarageQual"].fillna(df_train.GarageQual.mode()[0], inplace=True)

In [57]:
df_train["GarageQual"].isnull().sum()

In [58]:
df_train.GarageCond

In [59]:
df_train["GarageCond"].fillna(df_train.GarageCond.mode()[0], inplace=True)

In [60]:
df_train["GarageCond"].isnull().sum()

In [61]:
df_train.BsmtExposure

In [62]:
df_train["BsmtExposure"].fillna(df_train.BsmtExposure.mode()[0], inplace=True)

In [63]:
df_train["BsmtExposure"].isnull().sum()

In [64]:
df_train.BsmtFinType2

In [65]:
df_train["BsmtFinType2"].isnull().sum()

In [66]:
df_train["BsmtFinType2"].fillna(df_train.BsmtFinType2.mode()[0], inplace=True)

In [67]:
df_train["BsmtFinType2"].isnull().sum()

In [68]:
df_train.BsmtQual

In [69]:
df_train["BsmtQual"].isnull().sum()

In [70]:
df_train["BsmtQual"].fillna(df_train.BsmtQual.mode()[0], inplace=True)

In [71]:
df_train["BsmtQual"].isnull().sum()

In [72]:
df_train.BsmtCond

In [73]:
df_train["BsmtCond"].isnull().sum()

In [74]:
df_train["BsmtCond"].fillna(df_train.BsmtCond.mode()[0], inplace=True)

In [75]:
df_train["BsmtCond"].isnull().sum()

In [76]:
df_train.BsmtFinType1

In [77]:
df_train["BsmtFinType1"].isnull().sum()

In [78]:
df_train["BsmtFinType1"].fillna(df_train.BsmtFinType1.mode()[0], inplace=True)

In [79]:
df_train["BsmtFinType1"].isnull().sum()

In [80]:
df_train.MasVnrType

In [81]:
df_train.MasVnrArea

In [82]:
df_train["MasVnrArea"].isnull().sum()

In [83]:
ax = sns.distplot(df_train["MasVnrArea"])

In [84]:
df_train["MasVnrArea"].fillna(df_train.MasVnrArea.median(), inplace=True)

In [85]:
df_train["MasVnrArea"].isnull().sum()

In [86]:
df_train.MasVnrType

In [87]:
df_train.Electrical

In [88]:
df_train.isnull().sum()

In [89]:
df_train["Electrical"].fillna(df_train.Electrical.mode()[0], inplace=True)
df_train["MasVnrType"].fillna(df_train.MasVnrType.mode()[0], inplace=True)

In [90]:
df_train_nu = (df_train.isnull().sum() / len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu==0].index).sort_values(ascending=False)[:30]

In [91]:
missing_data = pd.DataFrame({"Missin Ratio" : df_train_nu})
missing_data.head()

In [92]:
df_test.info()

In [93]:
corrmat = df_train.corr()
plt.subplots(figsize=(20, 20))
sns.heatmap(corrmat, vmax=1.1, square=True)

In [94]:
df_test_nu = df_test.isnull().sum() / len(df_test)
df_test_nu = df_test_nu.drop(df_test_nu[df_test_nu==0].index)
missing_data_test = pd.DataFrame({"Missing Ratio" : df_test_nu})
missing_data_test.head()

In [96]:
df_test['LotFrontage'].fillna(df_test.LotFrontage.median(), inplace = True)
df_test['MasVnrArea'].fillna(df_test.MasVnrArea.median(), inplace = True)
df_test['TotalBsmtSF'].fillna(df_test.TotalBsmtSF.mode()[0], inplace = True)
df_test['BsmtFinSF1'].fillna(df_test.BsmtFinSF1.median(), inplace = True)
df_test['BsmtFinSF2'].fillna(df_test.BsmtFinSF2.mode()[0], inplace = True)
df_test['BsmtUnfSF'].fillna(df_test.BsmtUnfSF.median(), inplace = True)
df_test['GarageYrBlt'].fillna(df_test.GarageYrBlt.median(), inplace = True)
df_test['BsmtFullBath'].fillna(df_test.BsmtFullBath.mode()[0], inplace = True)
df_test['BsmtHalfBath'].fillna(df_test.BsmtHalfBath.mode()[0], inplace = True)
df_test['GarageCars'].fillna(df_test.GarageCars.mode()[0], inplace = True)
df_test['GarageArea'].fillna(df_test.GarageArea.mean(), inplace = True)

In [97]:
x_data = df_train.drop("SalePrice", axis=1)
y_data = df_train["SalePrice"]

Scaling

In [99]:
df_train.select_dtypes(include=["object"])

In [100]:
df_train_object = df_train.select_dtypes(include=["object"])

In [101]:
df_train_object.columns

In [102]:
from sklearn.preprocessing import LabelEncoder

In [103]:
object_cols = ('MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'c', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition')

LabelEncoder：文字にラベルをつける(0~N)、同じ文字＝同じID

In [104]:
for i in object_cols:
    lb_encoder = LabelEncoder()
    lb_encoder.fit(list(df_train[i].values))
    df_train[i] = lb_encoder.transform(list(df_train[i].values))

In [112]:
df_train.select_dtypes(include=["object"])

In [113]:
df_train.info()

Dealing with Test Set

In [114]:
df_test["BsmtFullBath"]

In [115]:
df_test["BsmtHalfBath"]

In [116]:
display(df_test["BsmtFinSF1"])
ax = sns.distplot(df_test["BsmtFinSF1"])

In [117]:
df_test["GarageCars"]

In [118]:
display(df_test["TotalBsmtSF"])
ax = sns.distplot(df_test["TotalBsmtSF"])

In [119]:
df_test["BsmtFinSF2"]

In [120]:
df_test["BsmtUnfSF"]

In [121]:
df_test_nu = (df_test.isnull().sum()/len(df_test)) * 100
df_test_nu = df_test_nu.drop(df_train_nu[df_test_nu==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_test_nu})
missing_data.head(100)

In [122]:
df_test.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1, inplace=True)

In [123]:
df_test['LotFrontage'].fillna(df_test.LotFrontage.median(), inplace = True)
df_test['GarageType'].fillna(df_test['GarageType'].mode()[0], inplace = True)
df_test['GarageYrBlt'].fillna(df_test.GarageYrBlt.median(), inplace = True)
df_test['GarageFinish'].fillna(df_test.GarageFinish.mode()[0], inplace = True)
df_test['GarageQual'].fillna(df_test.GarageQual.mode()[0], inplace = True)
df_test['GarageCond'].fillna(df_test.GarageCond.mode()[0], inplace = True)
df_test['BsmtExposure'].fillna(df_test.BsmtExposure.mode()[0], inplace = True)
df_test['BsmtFinType2'].fillna(df_test.BsmtFinType2.mode()[0], inplace = True)
df_test['BsmtQual'].fillna(df_test.BsmtQual.mode()[0], inplace = True)
df_test['BsmtCond'].fillna(df_test.BsmtCond.mode()[0], inplace = True)
df_test['BsmtFinType1'].fillna(df_test.BsmtFinType1.mode()[0], inplace = True)
df_test['MasVnrArea'].fillna(df_test.MasVnrArea.median(), inplace = True)
df_test['Electrical'].fillna(df_test.Electrical.mode()[0], inplace = True)
df_test['MasVnrType'].fillna(df_test.MasVnrType.mode()[0], inplace = True)
df_test['BsmtFullBath'].fillna(df_test.BsmtFullBath.mode()[0], inplace = True)
df_test['BsmtHalfBath'].fillna(df_test.BsmtHalfBath.mode()[0], inplace = True)
df_test['BsmtFinSF1'].fillna(df_test.BsmtFinSF1.median(), inplace = True)
df_test['GarageCars'].fillna(df_test.GarageCars.mode()[0], inplace = True)
df_test['GarageArea'].fillna(df_test.GarageArea.mean(), inplace = True)
df_test['TotalBsmtSF'].fillna(df_test.TotalBsmtSF.mode()[0], inplace = True)
df_test['BsmtFinSF2'].fillna(df_test.BsmtFinSF2.mode()[0], inplace = True)
df_test['BsmtUnfSF'].fillna(df_test.BsmtUnfSF.median(), inplace = True)
df_test['MSZoning'].fillna(df_test.MSZoning.mode()[0], inplace = True)
df_test['Functional'].fillna(df_test.Functional.mode()[0], inplace = True)
df_test['Utilities'].fillna(df_test.Utilities.mode()[0], inplace = True)
df_test['Exterior2nd'].fillna(df_test.Exterior2nd.mode()[0], inplace = True)
df_test['Exterior1st'].fillna(df_test.Exterior1st.mode()[0], inplace = True)
df_test['SaleType'].fillna(df_test.SaleType.mode()[0], inplace = True)
df_test['KitchenQual'].fillna(df_test.KitchenQual.mode()[0], inplace = True)

In [124]:
df_test_nu = (df_test.isnull().sum() / len(df_test)) * 100
df_test_nu = df_test_nu.drop(df_train_nu[df_test_nu == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :df_test_nu})
missing_data.head(100)

In [125]:
df_test.select_dtypes(include=["object"])
df_test_object = df_test.select_dtypes(include=["object"])
df_test_object.columns

In [128]:
for i in object_cols:
    lb_encoder = LabelEncoder()
    lb_encoder.fit(list(df_test[i].values))
    df_test[i] = lb_encoder.transform(list(df_test[i].values))

In [129]:
df_test.select_dtypes(include=["object"])
df_test.info()

In [130]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [131]:
x_data = df_train.drop("SalePrice", axis=1)
y_data = df_train["SalePrice"]

In [132]:
df_train.info()

In [133]:
x_data.info()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
x_train.info()

In [ ]:
x_data

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

In [ ]:
print("Mean Absolute Error(MAE) : ", metrics.mean_absolute_error(y_test, predictions))
print("Mean Squared Error(MSE) : ", metrics.mean_squared_error(y_test, predictions))
print("Root Mean Squared Error(RMSE) : ", np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("Explained Variance Score(EVS) : ", explained_variance_score(y_test, predictions))
print("R2 : ", metrics.r2_score(y_test, predictions))